In [64]:
import pandas as pd
import json
import re

In [13]:
table_names = pd.read_excel('/Users/ik/Downloads/2016_GCP_POA_for_NSW_short-header/Metadata/Metadata_2016_GCP_DataPack.xlsx', 
                     skiprows=9, sheet_name=0)

In [58]:
what_tables = list(table_names[table_names['Table name'].apply(lambda x: 'Total Personal Income' in x)]['Table number'])

In [16]:
cell_descr = pd.read_excel('/Users/ik/Downloads/2016_GCP_POA_for_NSW_short-header/Metadata/Metadata_2016_GCP_DataPack.xlsx', 
                     skiprows=10, sheet_name=1)

In [60]:
dsc = cell_descr[cell_descr['Profile table'].apply(lambda x: sum([table_name in x for table_name in what_tables]) > 0 )]

In [62]:
fls_cont_ = list(dsc['DataPack file'].unique())

In [164]:
age_groups = []

for lab in list(dsc['Short']):
    _ = re.search(r'_\d{2}_',lab)
    if _:
        age_groups.append(_.group(0)[:-1])
        
age_groups = sorted(list(set(age_groups)))

In [165]:
age_groups

['_15', '_20', '_25', '_35', '_45', '_55', '_65', '_75', '_85']

In [166]:
tbl = pd.concat([pd.read_csv(f'/Users/ik/Downloads/2016_GCP_POA_for_NSW_short-header/2016 Census GCP Postal Areas for NSW/2016Census_{t}_NSW_POA.csv').set_index('POA_CODE_2016') for t in fls_cont_], axis=1)

In [167]:
_ = tbl[tbl.index == 'POA2000']

In [168]:
ppl15 = tbl[[c for c in _.columns if ('_85' in c) and ('M' in c)]]

In [169]:
ppl15[ppl15.index == 'POA2000']

,M_Negtve_Nil_incme_85_yrs_ovr,M_1_149_85ov,M_150_299_85ov,M_300_399_85ov,M_400_499_85ov,M_500_649_85ov,M_650_799_85ov,M_800_999_85ov,M_1000_1249_85ov,M_1250_1499_85ov,M_1500_1749_85ov,M_1750_1999_85ov,M_2000_2999_85ov,M_3000_more_85ov,M_PI_NS_ns_85_yrs_ovr,M_Tot_85ov
POA_CODE_2016,,,,,,,,,,,,,,,,
POA2000,5,3,5,8,0,3,5,0,7,3,3,3,6,5,19,80


In [170]:
ppl15[ppl15.index == 'POA2000'].iloc[0,-1]

80

In [171]:
ppl15[ppl15.index == 'POA2000'].iloc[0,0:-1].sum()

75

In [172]:
income_dict = {}

In [173]:
ppl15[ppl15.index == 'POA2000'].to_dict(orient='index')['POA2000']

{'M_1000_1249_85ov': 7,
 'M_1250_1499_85ov': 3,
 'M_1500_1749_85ov': 3,
 'M_150_299_85ov': 5,
 'M_1750_1999_85ov': 3,
 'M_1_149_85ov': 3,
 'M_2000_2999_85ov': 6,
 'M_3000_more_85ov': 5,
 'M_300_399_85ov': 8,
 'M_400_499_85ov': 0,
 'M_500_649_85ov': 3,
 'M_650_799_85ov': 5,
 'M_800_999_85ov': 0,
 'M_Negtve_Nil_incme_85_yrs_ovr': 5,
 'M_PI_NS_ns_85_yrs_ovr': 19,
 'M_Tot_85ov': 80}

In [197]:
def is_income_col(col_name):
    
    gender, dollars_from, dollars_to = col_name.split('_')[:3]
    
    if not ((dollars_from.lower() in {'neg', 'negtve', 'pi'}) or dollars_from.isdigit()):
        return False
    
    if dollars_from.lower() in {'neg', 'negtve'}:
        dollars_from = '0'
    elif dollars_from.lower() == 'pi':
        dollars_from = '?'
        
    if dollars_to.lower() == 'nil':
        dollars_to = ''
    elif dollars_to.lower() == 'ns':
        dollars_to = ''
    elif dollars_to.lower() == 'more':
        dollars_to = '+'
    
    income = '$' + dollars_from + dollars_to
        
    return (gender.lower(), income) 

In [195]:
is_income_col('M_3000_more_85ov')

('M', '$3000+')